# Naver 영화 네티즌 평점 수집 - 미드웨이, 2019

In [233]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
url_base = 'https://movie.naver.com'
url_sub = '/movie/bi/mi/point.nhn?code=186615'
html = urlopen(url_base+url_sub)
soup = BeautifulSoup(html, 'html.parser')

### iframe의 src 주소의 값을 얻어야 함

In [234]:
ifr_url = soup.find('iframe')['src']
ifr_url

'/movie/bi/mi/pointWriteFormList.nhn?code=186615&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false'

In [235]:
url = url_base + ifr_url
html = urlopen(url)
soup = BeautifulSoup(html, 'html.parser')

In [236]:
count = soup.find('strong', 'total')
count

<strong class="total">관람객 평점 <em>4,813</em>건<button class="btn_review" id="open-form-btn">내 평점 등록</button></strong>

In [237]:
count = count.find('em').string
count

'4,813'

In [238]:
count = int(count.replace(',', ''))
count

4813

In [239]:
import math
loop_count = math.ceil(count/10)
loop_count

482

## 첫 번째 페이지에서 리뷰, 평점, 날짜 구해보기

### - 한 개의 리뷰에서 평점, 리뷰, 날짜 구해보기

In [240]:
results = soup.find(class_='score_result')

In [241]:
lis = results.find_all('li')
lis

[<li>
 <div class="star_score">
 <span class="st_off"><span class="st_on" style="width:90.0%"></span></span><em>9</em>
 </div>
 <div class="score_reple">
 <p>
 <!-- 스포일러 컨텐츠로 처리되는지 여부 -->
 <span id="_filtered_ment_0">
 														
 															
 															
 																미드웨이가 재밌으면 추천 백두산이 재밌으면 비추 
 															
 														
 														
 													</span>
 </p>
 <dl>
 <dt>
 <em>
 <a href="#" onclick="javascript:showPointListByNid(16564107, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
 <span>낄낄쿨레(sunj****)</span>
 </a>
 </em>
 <em>2019.12.31 09:48</em>
 </dt>
 <dd>
 <a class="go_report2" href="#" onclick="parent.clickcr(this, 'ara.report', '', '', event); common.report('false','sunj****', 'H1qGzChCI/HA8rusmPIDMX5pZzSyruuDrNXfeUzP7Iw=', '미드웨이가 재밌으면 추천 백두산이 재밌으면 비추 ', '16564107', 'point_after', false);return false;"><em>신고</em></a>
 </dd>
 </dl>
 </div>
 <div class="btn_area">
 <a class="_sympathyButton" href="#" onclick=

In [242]:
li = lis[0]
li

<li>
<div class="star_score">
<span class="st_off"><span class="st_on" style="width:90.0%"></span></span><em>9</em>
</div>
<div class="score_reple">
<p>
<!-- 스포일러 컨텐츠로 처리되는지 여부 -->
<span id="_filtered_ment_0">
														
															
															
																미드웨이가 재밌으면 추천 백두산이 재밌으면 비추 
															
														
														
													</span>
</p>
<dl>
<dt>
<em>
<a href="#" onclick="javascript:showPointListByNid(16564107, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
<span>낄낄쿨레(sunj****)</span>
</a>
</em>
<em>2019.12.31 09:48</em>
</dt>
<dd>
<a class="go_report2" href="#" onclick="parent.clickcr(this, 'ara.report', '', '', event); common.report('false','sunj****', 'H1qGzChCI/HA8rusmPIDMX5pZzSyruuDrNXfeUzP7Iw=', '미드웨이가 재밌으면 추천 백두산이 재밌으면 비추 ', '16564107', 'point_after', false);return false;"><em>신고</em></a>
</dd>
</dl>
</div>
<div class="btn_area">
<a class="_sympathyButton" href="#" onclick="parent.clickcr(this, 'ara.sym', 

In [243]:
score = li.find(class_='star_score').find('em').get_text()
score = int(score)
score

9

In [244]:
review = li.find(class_='score_reple').find('p').find('span').get_text()
review = review.strip(' \t\r\n')
review

'미드웨이가 재밌으면 추천 백두산이 재밌으면 비추'

In [225]:
review = li.find(class_='score_reple').find('p').find('span').string.strip(' \t\r\n')
review

'미드웨이가 재밌으면 추천 백두산이 재밌으면 비추'

In [245]:
li.find(class_='score_reple').find_all('em')

[<em>
 <a href="#" onclick="javascript:showPointListByNid(16564107, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
 <span>낄낄쿨레(sunj****)</span>
 </a>
 </em>,
 <em>2019.12.31 09:48</em>,
 <em>신고</em>]

In [246]:
date = li.find(class_='score_reple').find_all('em')[1].get_text()
date

'2019.12.31 09:48'

#### - 10개의 데이터에서 데이터 가져오기

In [288]:
point = []
review = []
date = []

for li in lis:
    point.append(int(li.find(class_='star_score').find('em').get_text()))
    count_span = len(li.find(class_='score_reple').find_all('span'))
    count_icon = len(li.find_all(class_='ico_viewer'))
    if count_span == 2:
        review.append(li.find(class_='score_reple').find('p').find('span').string.strip(' \t\r\n'))
    elif count_icon ==1:
        review.append(li.find(class_='score_reple').find('p').find_all('span')[1].string.strip(' \t\r\n'))
    else:
        review.append(li.find(class_='score_reple').find('a').string.strip(' \t\r\n'))
    date.append(li.find(class_='score_reple').find_all('em')[1].get_text())

In [247]:
li9 = lis[9]
li9

<li class="last">
<div class="star_score">
<span class="st_off"><span class="st_on" style="width:80.0%"></span></span><em>8</em>
</div>
<div class="score_reple">
<p>
<!-- 스포일러 컨텐츠로 처리되는지 여부 -->
<span id="_filtered_ment_9">
<span class="_unfold_ment" id="_unfold_ment9">
<a data-src="아니 에머리히 감독이 스토리에 고증을 철저히 지키다니 안믿긴다..(진짜 실제인물에 장면 하나하나가 거의 다 사실인건 솔직히 놀랍네요)(영화에 대한 팩트들)1.제작비는 500억이 아니라 1100억이고 2.소니가 진짜로 20년전에 미드웨이 영화화 반대했고3.평론가 평가가 안좋다?..진주만 보다 20점 높은 44점입니다.(전쟁 영화로써 좋은 점수입니다)4.고증이 안좋다?..스토리 고증은 정말 철저하고 전투기 등등의 디테일은 비슷하지만 완전 정확하진 않음5.중국이 나온다?:나옵니다..하지만 역사적 사실이고 일본은 당시에 미군을 도와준 중국을 보복했죠.(영화는 소니의 반대로 뒤늦게 20년만에 독립영화로 만들어졌는데 1100억의 제작비로 이정도로 강렬한 운명의 5분 미드웨이 해전을 보여준건 놀랍네요) " href="javascript:void(0);" onclick="unfoldPointMent(this);">
																		아니 에머리히 감독이 스토리에 고증을 철저히 지키다니 안믿긴다..(진짜 실제인물에 장면 하나하나가 거의 다 사실인건 솔직히 놀랍네요)(영화에 대한 팩트들)1.제작비는 500억이 아니라 1100억이고 2.소니가 진짜로...
																	</a>
</span>
</span>
</p>
<dl>
<dt>
<em>
<a href="#" onclick="javascript:showPointListByNid(16564157, 'a

In [248]:
li9.find(class_='score_reple').find('a').string.strip(' \t\r\n')

'아니 에머리히 감독이 스토리에 고증을 철저히 지키다니 안믿긴다..(진짜 실제인물에 장면 하나하나가 거의 다 사실인건 솔직히 놀랍네요)(영화에 대한 팩트들)1.제작비는 500억이 아니라 1100억이고 2.소니가 진짜로...'

In [254]:
len(li.find(class_='score_reple').find_all('span'))

2

In [283]:
li9.find(class_='score_reple').find_all('span')

[<span id="_filtered_ment_9">
 <span class="_unfold_ment" id="_unfold_ment9">
 <a data-src="아니 에머리히 감독이 스토리에 고증을 철저히 지키다니 안믿긴다..(진짜 실제인물에 장면 하나하나가 거의 다 사실인건 솔직히 놀랍네요)(영화에 대한 팩트들)1.제작비는 500억이 아니라 1100억이고 2.소니가 진짜로 20년전에 미드웨이 영화화 반대했고3.평론가 평가가 안좋다?..진주만 보다 20점 높은 44점입니다.(전쟁 영화로써 좋은 점수입니다)4.고증이 안좋다?..스토리 고증은 정말 철저하고 전투기 등등의 디테일은 비슷하지만 완전 정확하진 않음5.중국이 나온다?:나옵니다..하지만 역사적 사실이고 일본은 당시에 미군을 도와준 중국을 보복했죠.(영화는 소니의 반대로 뒤늦게 20년만에 독립영화로 만들어졌는데 1100억의 제작비로 이정도로 강렬한 운명의 5분 미드웨이 해전을 보여준건 놀랍네요) " href="javascript:void(0);" onclick="unfoldPointMent(this);">
 																		아니 에머리히 감독이 스토리에 고증을 철저히 지키다니 안믿긴다..(진짜 실제인물에 장면 하나하나가 거의 다 사실인건 솔직히 놀랍네요)(영화에 대한 팩트들)1.제작비는 500억이 아니라 1100억이고 2.소니가 진짜로...
 																	</a>
 </span>
 </span>,
 <span class="_unfold_ment" id="_unfold_ment9">
 <a data-src="아니 에머리히 감독이 스토리에 고증을 철저히 지키다니 안믿긴다..(진짜 실제인물에 장면 하나하나가 거의 다 사실인건 솔직히 놀랍네요)(영화에 대한 팩트들)1.제작비는 500억이 아니라 1100억이고 2.소니가 진짜로 20년전에 미드웨이 영화화 반대했고3.평론가 평가가 안좋다?..진주만 보다 20점 높은 44점입니다.(전쟁 영화로써 좋은 점수입니다)4.고증이 안좋다?..

In [255]:
len(li9.find(class_='score_reple').find_all('span'))

3

In [261]:
li4 = lis[4]
li4

<li>
<div class="star_score">
<span class="st_off"><span class="st_on" style="width:100.0%"></span></span><em>10</em>
</div>
<div class="score_reple">
<p>
<span class="ico_viewer">관람객</span>
<!-- 스포일러 컨텐츠로 처리되는지 여부 -->
<span id="_filtered_ment_4">
														
															
															
																교과서에는 미드웨이 해전에서 미국이 승리했다고 한 줄로 서술되어 있지만 단순히 한 줄로 설명할 수 없는 파란만장했던 전쟁. 
															
														
														
													</span>
</p>
<dl>
<dt>
<em>
<a href="#" onclick="javascript:showPointListByNid(16566092, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
<span>꼬깔콘의마술사(topm****)</span>
</a>
</em>
<em>2019.12.31 19:58</em>
</dt>
<dd>
<a class="go_report2" href="#" onclick="parent.clickcr(this, 'ara.report', '', '', event); common.report('false','topm****', 'vYXs5/PoWEG7HZjqZRu68uB646Q8PvyG0rMxbnWuNaY=', '교과서에는 미드웨이 해전에서 미국이 승리했다고 한 줄로 서술되어 있지만 단순히 한 줄로 설명할 수 없는 파란만장했던 전쟁. ', '16566092', 'point_after', false);return false;"><em>신고</em

In [282]:
li4.find(class_='score_reple').find_all('span')

[<span class="ico_viewer">관람객</span>,
 <span id="_filtered_ment_4">
 														
 															
 															
 																교과서에는 미드웨이 해전에서 미국이 승리했다고 한 줄로 서술되어 있지만 단순히 한 줄로 설명할 수 없는 파란만장했던 전쟁. 
 															
 														
 														
 													</span>,
 <span>꼬깔콘의마술사(topm****)</span>]

In [289]:
import pandas as pd
df = pd.DataFrame({'점수': point, '일시': date, '감상평': review})
df

,점수,일시,감상평
0,9,2019.12.31 09:48,미드웨이가 재밌으면 추천 백두산이 재밌으면 비추
1,10,2019.12.31 10:41,저 해전이 있었기에 우리나라 광복도 가능 했음
2,10,2019.12.31 09:38,백두산 상영관 대폭줄이고 미드웨이 상영관 대폭늘려라
3,10,2019.12.31 09:14,방금 개봉했는데 1점 준애는 뭐냐 ㅋㅋ 일본놈이냐? 이제 광고돌고 있을 시간이다.
4,10,2019.12.31 19:58,교과서에는 미드웨이 해전에서 미국이 승리했다고 한 줄로 서술되어 있지만 단순히 한 ...
5,10,2019.12.31 09:29,스케일 겁나커서 보는내내 입벌리고 봣네욬ㅋㅋ재밋습니다
6,10,2019.12.31 09:52,안보면후회함 백두산보다낫다
7,10,2019.12.31 09:01,재밌으면 추천 재미없으면 비추천 눌러주십쇼 형님들
8,10,2019.12.31 16:37,일본의 최대 실수는 미국을 건드린거다 그 대가로 원폭 처맞고 멸망함ㅋㅋㅋ지금 하는 ...
9,8,2019.12.31 10:14,아니 에머리히 감독이 스토리에 고증을 철저히 지키다니 안믿긴다..(진짜 실제인물에 ...


## 읽을 페이지 수 계산

In [292]:
count = soup.find('strong', 'total')
count

<strong class="total">관람객 평점 <em>4,813</em>건<button class="btn_review" id="open-form-btn">내 평점 등록</button></strong>

In [293]:
count = count.find('em').string
count

'4,813'

In [294]:
count = int(count.replace(',', ''))
count

4813

In [295]:
import math
total_page = math.ceil(count/10)
total_page

482

### 전체 리뷰에서 데이터 수집하기

In [298]:
point = []
review = []
date = []

for page in range(1, total_page+1):
    print(page, end = ' ')
    url = url_base + ifr_url + '&page=' + str(page)
    html = urlopen(url)
    soup = BeautifulSoup(html, 'html.parser')
    for li in lis:
        point.append(int(li.find(class_='star_score').find('em').get_text()))
        count_span = len(li.find(class_='score_reple').find_all('span'))
        count_icon = len(li.find_all(class_='ico_viewer'))
        if count_span == 2:
            review.append(li.find(class_='score_reple').find('p').find('span').string.strip(' \t\r\n'))
        elif count_icon ==1:
            review.append(li.find(class_='score_reple').find('p').find_all('span')[1].string.strip(' \t\r\n'))
        else:
            review.append(li.find(class_='score_reple').find('a').string.strip(' \t\r\n'))
        date.append(li.find(class_='score_reple').find_all('em')[1].get_text())

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

In [299]:
'관람객' in review

False

In [300]:
df = pd.DataFrame({'점수': point, '일시': date, '감상평': review})
df.tail(10)

,점수,일시,감상평
0,9,2019.12.31 09:48,미드웨이가 재밌으면 추천 백두산이 재밌으면 비추
1,10,2019.12.31 10:41,저 해전이 있었기에 우리나라 광복도 가능 했음
2,10,2019.12.31 09:38,백두산 상영관 대폭줄이고 미드웨이 상영관 대폭늘려라
3,10,2019.12.31 09:14,방금 개봉했는데 1점 준애는 뭐냐 ㅋㅋ 일본놈이냐? 이제 광고돌고 있을 시간이다.
4,10,2019.12.31 19:58,교과서에는 미드웨이 해전에서 미국이 승리했다고 한 줄로 서술되어 있지만 단순히 한 ...
...,...,...,...
4815,10,2019.12.31 09:29,스케일 겁나커서 보는내내 입벌리고 봣네욬ㅋㅋ재밋습니다
4816,10,2019.12.31 09:52,안보면후회함 백두산보다낫다
4817,10,2019.12.31 09:01,재밌으면 추천 재미없으면 비추천 눌러주십쇼 형님들
4818,10,2019.12.31 16:37,일본의 최대 실수는 미국을 건드린거다 그 대가로 원폭 처맞고 멸망함ㅋㅋㅋ지금 하는 ...


In [301]:
df.to_csv('data/midway.csv')